In [1]:
FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR   = '../Data_Sets/p3/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython import display
from ClassPerceptron import Perceptron
from sklearn import preprocessing
import grafica as gr
from ClassNeuronaGral import *
from NeuronaSoftMax import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
data = pd.read_csv(DATOS_DIR + 'Vinos.csv', sep= ';')

## Ejercicio 12

Utilice una red neuronal formada por una única capa de salida de 3 neuronas para clasificar las muestras de vino del archivo Vinos.csv descripto en el ejercicio anterior.

Realice 30 ejecuciones independientes utilizando el 60% y 80% de los ejemplos como entrenamiento y el resto como testeo.

Utilice un máximo de 400 iteraciones y velocidades de aprendizaje 0.1, 0.2 y 0.3.

Complete la siguiente tabla con los resultados de las siguientes configuraciones

### a) & b)

Función de activación ‘sigmoid’ y función de costo ‘ECM’ (error cuadrático medio)


In [ ]:

ejecuciones = 30
porcentajes = [0.6, 0.8]
alphas = [0.1, 0.2, 0.3]
N_ITER = 400
COTA = 1e-4
scaler = StandardScaler()
# Separar atributos y clase
X = data.iloc[:, 1:].values          # 13 atributos
X = scaler.fit_transform(X)
y1 = (data.iloc[:, 0].values == 1).astype(int)  # 1 si es tipo 1, 0 en caso contrario
y2 = (data.iloc[:, 0].values == 2).astype(int)  # 1 si es tipo 2, 0 en caso contrario
y3 = (data.iloc[:, 0].values == 3).astype(int)  # 1 si es tipo 3, 0 en caso contrario

resultados = {}

for alpha in alphas:
    resultados[alpha] = {}
    for p in porcentajes:
        resultados[alpha][p] = []
        for rep in range(ejecuciones): 
            # split train / test (en las 3 salidas)
            X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
                X, y1, y2, y3, train_size=p, shuffle=True
            )

            # entreno las 3 neuronas (one vs all)
            ngs = []
            for y_train in [y1_train, y2_train, y3_train]:
                ng = NeuronaGradiente(alpha=alpha, n_iter=N_ITER, cotaE=COTA, FUN='sigmoid')
                ng.fit(X_train, y_train)
                ngs.append(ng)

            # ---- PREDICCIÓN ----
            # Para cada neurona calculo probabilidad
            y1_prob = ngs[0].predict_nOut(X_test)
            y2_prob = ngs[1].predict_nOut(X_test)
            y3_prob = ngs[2].predict_nOut(X_test)

            # Probabilidades de las 3 neuronas
            Y_prob = np.column_stack([ngs[0].predict_nOut(X_test),
                                      ngs[1].predict_nOut(X_test),
                                      ngs[2].predict_nOut(X_test)])
            # Clase predicha = la neurona con mayor salida
            y_pred = np.argmax(Y_prob, axis=1) + 1

            # vector real (1,2,3) a partir de one-hot
            y_test_class = np.argmax(np.vstack([y1_test, y2_test, y3_test]).T, axis=1) + 1

            # ---- MÉTRICAS ----
            accuracy = np.mean(y_pred == y_test_class)

            # ECM multiclase
            Y_true = np.vstack([y1_test, y2_test, y3_test]).T
            ecm = np.mean((Y_prob - Y_true) ** 2)

            # Cross Entropy binaria por clase
            eps = np.finfo(float).eps
            cross_entropy = -np.mean(Y_true * np.log(Y_prob + eps) + (1 - Y_true) * np.log(1 - Y_prob + eps))

            resultados[alpha][p].append((accuracy, ecm, cross_entropy)) 

# impresión de resultados
for alpha in alphas:
    print(f"Resultados para alpha = {alpha}:")
    for p in porcentajes:
        acc_list  = [acc for acc, _, _ in resultados[alpha][p]]
        ecm_list  = [ecm for _, ecm, _ in resultados[alpha][p]]
        ce_list   = [ce  for _, _, ce in resultados[alpha][p]]

        print(f"  Porcentaje entrenamiento {int(p*100)}%:")
        print(f"    Accuracy -> Media = {np.mean(acc_list):.4f}, Desvío = {np.std(acc_list):.4f}")
        print(f"    ECM      -> Media = {np.mean(ecm_list):.6f}, Desvío = {np.std(ecm_list):.6f}")
        print(f"    CE bin   -> Media = {np.mean(ce_list):.6f}, Desvío = {np.std(ce_list):.6f}")

Resultados para alpha = 0.1:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9514, Desvío = 0.0205
    ECM      -> Media = 0.031823, Desvío = 0.009688
    CE bin   -> Media = 0.163252, Desvío = 0.058385
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9481, Desvío = 0.0266
    ECM      -> Media = 0.029526, Desvío = 0.011668
    CE bin   -> Media = 0.147429, Desvío = 0.086654
Resultados para alpha = 0.2:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9505, Desvío = 0.0217
    ECM      -> Media = 0.033059, Desvío = 0.009412
    CE bin   -> Media = 0.174692, Desvío = 0.069746
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9472, Desvío = 0.0339
    ECM      -> Media = 0.031003, Desvío = 0.016863
    CE bin   -> Media = 0.187480, Desvío = 0.147863
Resultados para alpha = 0.3:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9505, Desvío = 0.0195
    ECM      -> Media = 0.030955, Desvío = 0.008411
    CE bin   -> Media = 0.193522, Desvío = 0.

### c) 

Función de activación ‘tanh’ y función de costo ‘ECM’ (error cuadrático medio)

In [10]:
# Separar atributos y clase
X = data.iloc[:, 1:].values          # 13 atributos
X = scaler.fit_transform(X)
y1 = (data.iloc[:, 0].values == 1).astype(int)  # 1 si es tipo 1, 0 en caso contrario
y2 = (data.iloc[:, 0].values == 2).astype(int)  # 1 si es tipo 2, 0 en caso contrario
y3 = (data.iloc[:, 0].values == 3).astype(int)  # 1 si es tipo 3, 0 en caso contrario

#Preparamos las Ys para one-hot 
y1 = 2 * y1 - 1  # 1 si es tipo 1, -1 en caso contrario
y2 = 2 * y2 - 1  # 1 si es tipo 2, -1 en caso contrario
y3 = 2 * y3 - 1  # 1 si es tipo 3, -1 en caso contrario

resultados = {}

for alpha in alphas:
    resultados[alpha] = {}
    for p in porcentajes:
        resultados[alpha][p] = []
        for rep in range(ejecuciones): 
            # split train / test (en las 3 salidas)
            X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
                X, y1, y2, y3, train_size=p, shuffle=True
            )

            # entreno las 3 neuronas (one vs all)
            ngs = []
            for y_train in [y1_train, y2_train, y3_train]:
                ng = NeuronaGradiente(alpha=alpha, n_iter=N_ITER, cotaE=COTA, FUN='tanh')
                ng.fit(X_train, y_train)
                ngs.append(ng)

            # ---- PREDICCIÓN ----
            # Para cada neurona calculo probabilidad
            y1_prob = ngs[0].predict_nOut(X_test)
            y2_prob = ngs[1].predict_nOut(X_test)
            y3_prob = ngs[2].predict_nOut(X_test)

            # Probabilidades de las 3 neuronas
            Y_prob = np.column_stack([ngs[0].predict_nOut(X_test),
                                      ngs[1].predict_nOut(X_test),
                                      ngs[2].predict_nOut(X_test)])
            # Clase predicha = la neurona con mayor salida
            y_pred = np.argmax(Y_prob, axis=1) + 1

            # vector real (1,2,3) a partir de one-hot
            y_test_class = np.argmax(np.vstack([y1_test, y2_test, y3_test]).T, axis=1) + 1

            # ---- MÉTRICAS ----
            accuracy = np.mean(y_pred == y_test_class)

            # ECM multiclase
            Y_true = np.vstack([y1_test, y2_test, y3_test]).T
            ecm = np.mean((Y_prob - Y_true) ** 2)

            resultados[alpha][p].append((accuracy, ecm)) 

# impresión de resultados
for alpha in alphas:
    print(f"Resultados para alpha = {alpha}:")
    for p in porcentajes:
        acc_list  = [acc for acc, _ in resultados[alpha][p]]
        ecm_list  = [ecm for _, ecm, in resultados[alpha][p]]

        print(f"  Porcentaje entrenamiento {int(p*100)}%:")
        print(f"    Accuracy -> Media = {np.mean(acc_list):.4f}, Desvío = {np.std(acc_list):.4f}")
        print(f"    ECM      -> Media = {np.mean(ecm_list):.6f}, Desvío = {np.std(ecm_list):.6f}")

Resultados para alpha = 0.1:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9486, Desvío = 0.0230
    ECM      -> Media = 0.132335, Desvío = 0.045392
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9417, Desvío = 0.0315
    ECM      -> Media = 0.138357, Desvío = 0.070067
Resultados para alpha = 0.2:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9519, Desvío = 0.0151
    ECM      -> Media = 0.127380, Desvío = 0.031704
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9500, Desvío = 0.0308
    ECM      -> Media = 0.119255, Desvío = 0.060887
Resultados para alpha = 0.3:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9458, Desvío = 0.0181
    ECM      -> Media = 0.134051, Desvío = 0.035531
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9472, Desvío = 0.0262
    ECM      -> Media = 0.128438, Desvío = 0.061917


### d) 

Capa ‘Softmax’ y función de costo ‘EC’ (entropía cruzada).

Pdt:(esto son 3 neuronas con gradiente binario, no una red multiclase)

In [ ]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # estabilidad numérica
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)


# Separar atributos y clase
X = data.iloc[:, 1:].values          # 13 atributos
X = scaler.fit_transform(X)
y1 = (data.iloc[:, 0].values == 1).astype(int)  # 1 si es tipo 1, 0 en caso contrario
y2 = (data.iloc[:, 0].values == 2).astype(int)  # 1 si es tipo 2, 0 en caso contrario
y3 = (data.iloc[:, 0].values == 3).astype(int)  # 1 si es tipo 3, 0 en caso contrario

resultados = {}

for alpha in alphas:
    resultados[alpha] = {}
    for p in porcentajes:
        resultados[alpha][p] = []
        for rep in range(ejecuciones): 
            # split train / test (en las 3 salidas)
            X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(
                X, y1, y2, y3, train_size=p, shuffle=True
            )

            # entreno las 3 neuronas (one vs all)
            ngs = []
            for y_train in [y1_train, y2_train, y3_train]:
                ng = NeuronaGradiente(alpha=alpha, n_iter=N_ITER, cotaE=COTA, FUN='sigmoid')
                ng.fit(X_train, y_train)
                ngs.append(ng)

            # ---- PREDICCIÓN ----
            # Obtener salidas crudas (antes del softmax)
            Y_raw = np.column_stack([
                ngs[0].predict_nOut(X_test),
                ngs[1].predict_nOut(X_test),
                ngs[2].predict_nOut(X_test)
            ])

            # Aplicar softmax a las 3 neuronas
            Y_prob = softmax(Y_raw)

            # Clase predicha = argmax de la probabilidad softmax
            y_pred = np.argmax(Y_prob, axis=1) + 1

            # vector real (1,2,3) a partir de one-hot
            y_test_class = np.argmax(np.vstack([y1_test, y2_test, y3_test]).T, axis=1) + 1

            # ---- MÉTRICAS ----
            accuracy = np.mean(y_pred == y_test_class)

            # ECM multiclase
            Y_true = np.vstack([y1_test, y2_test, y3_test]).T
            ecm = np.mean((Y_prob - Y_true) ** 2)

            # Entropía cruzada multiclase (categorical CE)
            eps = np.finfo(float).eps
            cross_entropy = -np.mean(np.sum(Y_true * np.log(Y_prob + eps), axis=1))

            resultados[alpha][p].append((accuracy, ecm, cross_entropy)) 




# impresión de resultados
for alpha in alphas:
    print(f"Resultados para alpha = {alpha}:")
    for p in porcentajes:
        acc_list  = [acc for acc, _, _ in resultados[alpha][p]]
        ecm_list  = [ecm for _, ecm, _ in resultados[alpha][p]]
        ce_list   = [ce  for _, _, ce in resultados[alpha][p]]

        print(f"  Porcentaje entrenamiento {int(p*100)}%:")
        print(f"    Accuracy -> Media = {np.mean(acc_list):.4f}, Desvío = {np.std(acc_list):.4f}")
        print(f"    ECM      -> Media = {np.mean(ecm_list):.6f}, Desvío = {np.std(ecm_list):.6f}")
        print(f"    CE    -> Media = {np.mean(ce_list):.6f}, Desvío = {np.std(ce_list):.6f}")

Resultados para alpha = 0.1:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9458, Desvío = 0.0249
    ECM      -> Media = 0.105875, Desvío = 0.005730
    CE bin   -> Media = 0.615922, Desvío = 0.022788
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9565, Desvío = 0.0318
    ECM      -> Media = 0.104318, Desvío = 0.006572
    CE bin   -> Media = 0.610006, Desvío = 0.026797
Resultados para alpha = 0.2:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9528, Desvío = 0.0198
    ECM      -> Media = 0.103760, Desvío = 0.003659
    CE bin   -> Media = 0.607327, Desvío = 0.014812
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9611, Desvío = 0.0283
    ECM      -> Media = 0.102749, Desvío = 0.006230
    CE bin   -> Media = 0.603494, Desvío = 0.025172
Resultados para alpha = 0.3:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9542, Desvío = 0.0183
    ECM      -> Media = 0.103952, Desvío = 0.003778
    CE bin   -> Media = 0.608524, Desvío = 0.

Prueba con cross-entropy categórica (no binaria).

In [2]:
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values - 1   # clases 0,1,2

scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encoding
Y = np.zeros((len(y), 3))
Y[np.arange(len(y)), y] = 1

# ---------- Experimentos ----------
alphas = [0.1, 0.2, 0.3]
porcentajes = [0.6, 0.8]
ejecuciones = 10
N_ITER = 2000
COTA = 1e-5

resultados = {}

for alpha in alphas:
    resultados[alpha] = {}
    for p in porcentajes:
        accs, ecs, ces = [], [], []
        for _ in range(ejecuciones):
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=p, shuffle=True)

            ng = NeuronaSoftmax(alpha=alpha, n_iter=N_ITER, cotaE=COTA)
            ng.fit(X_train, Y_train)

            Y_prob = ng.predict_proba(X_test)
            y_pred = np.argmax(Y_prob, axis=1)
            y_true = np.argmax(Y_test, axis=1)

            # Accuracy
            acc = np.mean(y_pred == y_true)

            # ECM
            ecm = np.mean((Y_prob - Y_test) ** 2)

            # Cross entropy categórica
            ce = -np.mean(np.sum(Y_test * np.log(Y_prob + 1e-15), axis=1))

            accs.append(acc)
            ecs.append(ecm)
            ces.append(ce)

        resultados[alpha][p] = (np.mean(accs), np.std(accs),
                                np.mean(ecs), np.std(ecs),
                                np.mean(ces), np.std(ces))


# ---------- Resultados ----------
for alpha in alphas:
    print(f"\nResultados para alpha = {alpha}:")
    for p in porcentajes:
        acc_mean, acc_std, ecm_mean, ecm_std, ce_mean, ce_std = resultados[alpha][p]
        print(f"  Porcentaje entrenamiento {int(p*100)}%:")
        print(f"    Accuracy -> Media = {acc_mean:.4f}, Desvío = {acc_std:.4f}")
        print(f"    ECM      -> Media = {ecm_mean:.6f}, Desvío = {ecm_std:.6f}")
        print(f"    CE cat   -> Media = {ce_mean:.6f}, Desvío = {ce_std:.6f}")


Resultados para alpha = 0.1:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9361, Desvío = 0.0198
    ECM      -> Media = 0.032705, Desvío = 0.007132
    CE cat   -> Media = 0.289365, Desvío = 0.055233
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9500, Desvío = 0.0272
    ECM      -> Media = 0.022269, Desvío = 0.011616
    CE cat   -> Media = 0.136798, Desvío = 0.086729

Resultados para alpha = 0.2:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9292, Desvío = 0.0313
    ECM      -> Media = 0.035851, Desvío = 0.011316
    CE cat   -> Media = 0.278094, Desvío = 0.127218
  Porcentaje entrenamiento 80%:
    Accuracy -> Media = 0.9472, Desvío = 0.0262
    ECM      -> Media = 0.029554, Desvío = 0.014118
    CE cat   -> Media = 0.281098, Desvío = 0.250956

Resultados para alpha = 0.3:
  Porcentaje entrenamiento 60%:
    Accuracy -> Media = 0.9208, Desvío = 0.0335
    ECM      -> Media = 0.041438, Desvío = 0.017609
    CE cat   -> Media = 0.323382, Desvío =